In [1]:
# get logistic regression module
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import Snippets.log_regression as lr

# other libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load the Spambase-Data set
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data', sep=',', header=None)

In [38]:
# Split dataframe into X_train, X_test, y_train, y_test
X = df.iloc[:, :-1] 
y = df.iloc[:, -1] 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [218]:
# Standardize
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

In [10]:
def count_FP(y_true, y_predict):
    FP = 0;
    for i, elem in enumerate(y_true):
        if elem == 0 and y_predict[i] == 1:
            FP += 1
    return FP

In [11]:
def count_FN(y_true, y_predict):
    FN = 0;
    for i, elem in enumerate(y_true):
        if elem == 1 and y_predict[i] == 0:
            FN += 1
    return FN

In [12]:
# Calculate the Fß measure for ß = (.5, 1)
def get_F_measure(beta, y_true, y_predict):
    TP = len(y_true) - sum(abs(y_true-y_predict))
    FP = count_FP(y_true, y_predict)
    FN = count_FN(y_true, y_predict)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F = (1+beta**2) * (precision * recall) / (beta**2 * precision + recall)
    return F

In [8]:
beta = lr.get_coefficients(X_train, y_train, steps=50000, gamma=5e-5, method='gradient')

In [13]:
y_predict = lr.predict(X_test, beta)
print("F-Measure (beta = 0.5): {}".format(get_F_measure(0.5, y_test, y_predict)))
print("F-Measure (beta = 1): {}".format(get_F_measure(1, y_test, y_predict)))

F-Measure (beta = 0.5): 0.957205504309693
F-Measure (beta = 1): 0.9565545901020023


### Logistic ridge regression

In [324]:
def basis(X, mu, lambd, d):
    X_transform = []
    for col in pd.DataFrame(X).columns:
        kappa_values = []
        for i in range(d):
            kap = kappa(X[col], mu[i], lambd)
            kappa_values.append(kap)
        X_transform.append(kappa_values)
    print(np.asarray(X_transform).shape)
    return np.asarray(X_transform)

In [325]:
def kappa(X, mu, lambd):
    L2 = np.linalg.norm(X-mu)
    kap = np.exp(-1/lambd*L2)
     
    return kap

In [326]:
def ridge_get_coefficients(X_train, y_train, delta):
    #pad by one (I should really make a function for that)
    X_train = np.hstack((np.ones((X_test.shape[0],1)), X_test)) 
    
    # Test parameters
    mu = [0.1,0.2,0.3,0.4]
    d = len(mu)
    lambd=1
    
    # Müsste X_train.T nicht auch mit der basis function sein? --> Ergebnisse nachher mal checken
    beta = np.dot(np.linalg.inv(np.dot(basis(X_train, mu, lambd, d).T , basis(X_train, mu, lambd, d)) + delta**2*np.identity(X_train.shape[1])), X_train.T)*y_train.values
    #beta= np.dot(np.linalg.inv(np.dot(X_train.T, X_train) +  delta**2*np.identity(X_train.shape[1])), X_train.T)*y_train.values
    
    return beta
    

In [327]:
beta = ridge_get_coefficients(X_train, y_train, 1e-5)

(58, 4)
(58, 4)


ValueError: operands could not be broadcast together with shapes (4,4) (58,58) 

In [247]:
copy = X_train.copy()
copy = np.hstack((np.ones((X_test.shape[0],1)), X_test)) 
delta = 1e-3
#beta = np.dot(np.linalg.inv(copy)* basis(copy) + delta**2*np.identity(copy.shape[1])), copy.T)*y_train.values
#beta


In [301]:
copy_dot = np.dot(copy.T, copy)
a = np.dot(np.linalg.inv(copy_dot + delta**2*np.identity(copy_dot.shape[0])), copy.T)
a.shape
copy.T.shape
np.dot(a.T, copy.T).shape

basis(copy, d=3, lambd=1, mu=[1,2,3])
#kappa(copy, 2, 1)

#L2 = np.linalg.norm(copy-1)

#np.dot(np.linalg.inv(np.dot(X_train.T, X_train) +  delta**2*np.identity(X_train.shape[1])), X_train.T)*y_train.values

(3,)